In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os

# Question 1 : Video to Image and vice-versa

## 1.1 Capture frames from video

In [2]:
def Capture_Frames(path,output_dir,frameRate):
    
    # Path to video file caputers into object 
    vidObj = cv.VideoCapture(path)
    
    # Used as counter variable 
    count = 0
  
    # checks whether frames were extracted 
    success = 1
    sec = 1
    while success: 
          
        vidObj.set(cv.CAP_PROP_POS_MSEC,sec*1000)
        
        # vidObj object calls read function extract frames
        success,image = vidObj.read()
        if not success:
            print("Can't receive frame. Exiting ...")
            break
    
        # Saves the frames with frame-count 
        # save frame as JPG file
        cv.imwrite(os.path.join(output_dir, "image_"+str(count)+".jpg"), image)
        
        count+=1
        sec = sec + frameRate
        sec = round(sec, 2)

In [3]:
if __name__ == '__main__': 
  
    # Calling the function 
    path = "input_data\\question1_1\\video1.mp4"
    output_dir = "output_data\\video_to_frame"
    Capture_Frames(path,output_dir,frameRate=1) 
    

Can't receive frame. Exiting ...


## 1.2 Converting frames to video

In [5]:
def Frames_to_Video(input_dir,output_dir,fps):
    img_array = []
    
    # Taking each image and forming array of images
    for count in range(len(os.listdir(path))):
        filename = path+'/image_'+str(count)+'.jpg'
        img = cv.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)
 
    # Defining the path and filename to save with fps and size(w*h)
    vidoutObj = cv.VideoWriter(os.path.join(output_dir,'project.avi'),cv.VideoWriter_fourcc(*'DIVX'), fps, size)
 
    # each image in image_array is written to videowriter object
    for i in range(len(img_array)):
        vidoutObj.write(img_array[i])
    vidoutObj.release()

In [6]:
if __name__ == '__main__': 
  
    # Calling the function 
    path = "input_data\\question1_2"
    output_dir = "output_data\\frame_to_video"
    Frames_to_Video(path,output_dir,fps=30) 
    

# Question 2 : Capturing Images from webcam

In [9]:
def Cam_Capture_Frames(output_dir,frameRate):
    capture = cv.VideoCapture(0)

    if not capture.isOpened():
        print("Cannot open camera")
        exit()
    count = 0
    success = True
    print('Press q to exit')
    sec = 1
    while success:
        
        capture.set(cv.CAP_PROP_POS_MSEC,sec*1000)
        # Capture frame-by-frame
        success, frame = capture.read()
        # if frame is read correctly ret is True
        if not success:
            print("Can't receive frame. Exiting ...")
            break
        
        # Display the resulting frame
        cv.imwrite(os.path.join(output_dir, "image_"+str(count)+".jpg"), frame)
        cv.imshow('Frame', frame)
        if cv.waitKey(1) == ord('q'):
            break
        count+=1
        sec = sec + frameRate
        sec = round(sec, 2)
    # When everything done, release the capture
    capture.release()
    cv.destroyAllWindows()

In [12]:
if __name__ == '__main__': 
  
    # Calling the function 
    output_dir = "output_data\\frame_from_cam"
    Cam_Capture_Frames(output_dir,frameRate=0.01) 

Press q to exit


# Question 3 : Chroma Keying

In [13]:
def chroma_keying(bgpath,fgpath,output_dir):
    img_array = []
    # Reading two videofile fg and bg
    bg = cv.VideoCapture(bgpath)
    fg = cv.VideoCapture(fgpath)

    success,bg_frame = bg.read()
    ret,fg_frame = fg.read()

    count = 0
    success = True
    
    print('Press q to exit')
    while success:
        success,bg_frame = bg.read()
        ret,fg_frame = fg.read()
        
        if not success or not ret:
            print("Can't receive frame. Exiting ...")
            break
            
        bg_frame = cv.resize(bg_frame,(640,360))
        fg_frame = cv.resize(fg_frame,(640,360))
        
        # converting bgr to hsv color space
        hsv = cv.cvtColor(bg_frame,cv.COLOR_BGR2HSV)
        
        # defining green color range and creating a mask
        lower_green = np.array([50,150,10])
        upper_green = np.array([70,255,255])
        mask1 = cv.inRange(hsv, lower_green, upper_green)
        
        # inverting the mask to remove detected green color
        mask2 = cv.bitwise_not(mask1)
        
        # bitwise anding bg_frame with inverted mask to remove green color from frame
        res1 = cv.bitwise_and(bg_frame,bg_frame,mask=mask2)
        
        # bitwise anding fg_frame with mask of green color detected to replace green color pixels with fg_frame pixels 
        res2 = cv.bitwise_and(fg_frame, fg_frame, mask = mask1)
        
        # adding both bg and fg frames together
        output = cv.addWeighted(res1,1,res2,1,0)

        cv.imshow('frame',output)
        
        height, width, layers = output.shape
        size = (width,height)
        img_array.append(output)
        
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
    
    vidoutObj = cv.VideoWriter(os.path.join(output_dir,'chroma_keying.avi'),cv.VideoWriter_fourcc(*'DIVX'),30, size)
    for i in range(len(img_array)):
        vidoutObj.write(img_array[i])
    vidoutObj.release()
    # Release everything if job is finished
    bg.release()
    fg.release()
    cv.destroyAllWindows()

In [14]:
if __name__ == '__main__': 
  
    # Calling the function 
    fgpath = "input_data\\question3\\fg.mp4"
    bgpath = "input_data\\question3\\bg.mp4"
    output_dir = "output_data\chroma_keying"
    chroma_keying(bgpath,fgpath,output_dir) 

Press q to exit
Can't receive frame. Exiting ...


# Question 4 : Face Detection 

In [15]:
def Face_Detection():

    # Load the cascade
    face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')

    # To capture video from webcam. 
    capture = cv.VideoCapture(0)

    while True:
        # Read the frame
        _, img = capture.read()
        # Convert to grayscale
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        # Detect the faces
        faces = face_cascade.detectMultiScale(gray, 2, 4)
        # Draw the rectangle around each face
        for (x, y, w, h) in faces:
            cv.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
        # Display
        cv.imshow('img', img)
        # Stop if escape key is pressed
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
    # Release the VideoCapture object
    capture.release()

In [16]:
if __name__ == '__main__': 
  
    # Calling the function 
    Face_Detection()